In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 3.4 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 24.8 MB/s 
     |████████████████████████████████| 1.2 MB 47.0 MB/s 
     |████████████████████████████████| 61 kB 571 kB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=bed3d5b8b713fa06e7494bfa061dd71a6e23f1e7e108fecc1e6aff9a2464c4e9
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)
test
1
2


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=512)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 2139, TN: 1798, FP: 769, FN: 293
precision, recall: [0.7355570839064649, 0.8795230263157895]


In [ ]:
#cross-encoder/ms-marco-TinyBERT-L-2-v2, temp=alles, epoch = 3
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

TP: 2866, TN: 8103, FP: 1880, FN: 7143
precision, recall: [0.6038769490096924, 0.28634229193725647]


In [ ]:
#cross-encoder/ms-marco-TinyBERT-L-2-v2, temp=alles, epoch = 1
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=1, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-1c8fffe8-e933-d0ec-d9ff-5025898cc0a6)


Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

TP: 3763, TN: 7960, FP: 2023, FN: 6246
precision, recall: [0.6503629450397511, 0.37596163452892395]


In [3]:
#cross-encoder/ms-marco-TinyBERT-L-2-v2, temp=alles, epoch = 1, length = 512
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=1, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-30087a54-4c86-29c5-daf9-1fd40601f2cb)


Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

TP: 3604, TN: 8248, FP: 1735, FN: 6405
precision, recall: [0.6750327776737217, 0.36007593166150464]


In [ ]:
#cross-encoder/qnli-distilroberta-base, untrained
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=512)


# Does this need more training?
#model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

TP: 4129, TN: 6292, FP: 3691, FN: 5880
precision, recall: [0.5280051150895141, 0.4125287241482666]


In [ ]:
#cross-encoder/qnli-distilroberta-base, volledige dataset, 1 epoch
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=512)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=1, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

TP: 6730, TN: 7199, FP: 2784, FN: 3279
precision, recall: [0.7073785999579567, 0.6723948446398241]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3, max_length=256
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3")

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 1685, TN: 2236, FP: 331, FN: 747
precision, recall: [0.8358134920634921, 0.6928453947368421]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
    pairs.append([str(row[2]),str(row[3])])
    labels.append(int(row[4]))


#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 1894, TN: 2061, FP: 506, FN: 538
precision, recall: [0.7891666666666667, 0.7787828947368421]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=10000, epoch = 3, max_length=256, eval = CEBinaryAccuracyEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 10000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
    pairs.append([str(row[2]),str(row[3])])
    labels.append(int(row[4]))


#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryAccuracyEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-47eb85c6-0e5c-aa2f-c0d8-921e9d02b649)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

TP: 4028, TN: 4156, FP: 928, FN: 887
precision, recall: [0.8127522195318806, 0.8195320447609359]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3, max_length=256, eval = CEBinaryAccuracyEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
    pairs.append([str(row[2]),str(row[3])])
    labels.append(int(row[4]))


#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryAccuracyEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-4a7e8468-3f83-f029-51c8-9c11b50ab20e)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 1384, TN: 2335, FP: 232, FN: 1048
precision, recall: [0.8564356435643564, 0.569078947368421]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000 training maar precision/recall op volledige set, epoch = 3, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
all_examples = []
pairs = []
labels = []
temp = 1
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    if temp < 5000:
      temp += 1
      #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
      # We should take this into account by giving a small penalty to label based on position
      train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
      pairs.append([str(row[2]),str(row[3])])
      labels.append(int(row[4]))

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-2942ac33-a38d-7b26-a92b-b807dad55ac7)


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 3067, TN: 8819, FP: 1164, FN: 6942
precision, recall: [0.7248877333963601, 0.30642421820361676]


In [3]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
all_examples = []
pairs = []
labels = []
temp = 1
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    if temp < 5000:
      temp += 1
      #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
      # We should take this into account by giving a small penalty to label based on position
      train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
      pairs.append([str(row[2]),str(row[3])])
      labels.append(int(row[4]))

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-fc5f79ce-4ead-8df5-4063-3422387b6c9b)


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 1885, TN: 2058, FP: 509, FN: 547
precision, recall: [0.7873851294903926, 0.7750822368421053]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=alles, epoch = 3, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 99999999:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
all_examples = []
pairs = []
labels = []
temp = 1
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    if temp < 9999999999:
      temp += 1
      #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
      # We should take this into account by giving a small penalty to label based on position
      train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
      pairs.append([str(row[2]),str(row[3])])
      labels.append(int(row[4]))

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels) )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-2942ac33-a38d-7b26-a92b-b807dad55ac7)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

TP: 8124, TN: 8306, FP: 1677, FN: 1885
precision, recall: [0.8288950107131925, 0.8116694974522929]


In [ ]:
#cross-encoder/qnli-distilroberta-base, temp=5000, epoch = 3, max_length=256, eval = CEBinaryClassificationEvaluator, optimizer = Rprop
import sentence_transformers.cross_encoder.evaluation
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
    pairs.append([str(row[2]),str(row[3])])
    labels.append(int(row[4]))


#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=3, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/qnli_distilroberta_base_5000_3",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels),optimizer_class= torch.optim.Rprop )

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-4a7e8468-3f83-f029-51c8-9c11b50ab20e)


Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/313 [00:00<?, ?it/s]

TP: 1260, TN: 1858, FP: 709, FN: 1172
precision, recall: [0.6399187404773997, 0.5180921052631579]


In [ ]:
#Runs model trained on full dataset on dev_data.csv
import sentence_transformers.cross_encoder.evaluation
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/dev_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
            if len(row) != 5:
                print("error in input")
                print(row)
            else:
                #TODO: verwijderen als we volledige dataset inlezen
                if temp < 5000:
                    temp += 1
                    dev_data_dict.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    # Make dictionaries containing {key: queryNumber, value: [documentNumber]} but separate dicts for labels 0 and 1
    # Used later for recall/precision
    if int(row[0]) not in dev_data_1:
        dev_data_1[int(row[0])] = []
    if int(row[0]) not in dev_data_0:
        dev_data_0[int(row[0])] = []
    if int(row[4]) == 1:
        dev_data_1[int(row[0])].append(int(row[1]))
    else:
        dev_data_0[int(row[0])].append(int(row[1]))
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[2]),str(row[3])],label=float(row[4]) ) )
    pairs.append([str(row[2]),str(row[3])])
    labels.append(int(row[4]))


#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('./drive/MyDrive/modelOutput/Full', max_length=256)

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in dev_data_dict])

output_binary_label_unsorted = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

#print(sorted([[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))], key=lambda x: (x[0], -x[2], x[1])))
#scores_query = [[dev_data_dict[i][0], dev_data_dict[i][1], math.floor(scores[i] + 0.5)] for i in range(len(dev_data_dict))]

# TODO: use same output format as dev_data.csv, label should probably be 1 or 0 instead of score

# TODO: actually sort results per query based on scores

#scores_sorted = sorted(scores_query, key=lambda x: (x[0], -x[2], x[1]))

#expected_scores = [[row[0], row[1], int(row[4])] for row in dev_data_dict]
#expected_scores = sorted(expected_scores, key=lambda x: (x[0], -x[2], x[1]))

# Based on code from https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
def rbo(in1, in2, p=0.9):
    # tail recursive helper function
    list1 = [row[1] for row in in1]
    list2 = [row[1] for row in in2]
    def helper(ret, j, start, d):
        result = ret
        for i in range(j, d + 1):
            l1 = set(list1[start:start+i]) if start + i < len(list1) else set(list1[start:])
            l2 = set(list2[start:start+i]) if start + i < len(list2) else set(list2[start:])
            a_d = len(l1.intersection(l2))/i
            term = math.pow(p, i) * a_d
            result = ret + term
        return result
    result_list = []
    start = 0
    k = 0
    current_query = in1[0][0]
    for i in range(len(list1)):
        if current_query != in1[i][0] or i == len(list1) - 1:
            x_k = len(set(list1[start:start+k]).intersection(set(list2[start:start+k])))
            summation = helper(0, 1, start, k)
            result_list.append(((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation))
            start = k
            current_query = in1[i][0]
        k += 1
    return sum(result_list) / len(result_list)

def precision_recall(dict_0,dict_1,results):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    # precision = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal resultaten met label 1)
    # recall = (aantal resultaten met label 1 die in testdata ook label 1 hebben)/(aantal in testdata met label 1)
    for row in results:
        # positive
        # If label is 1 (positive)
        if int(row[2]) == 1:
            # If document in this positive result is indeed in positive results for this query
            if int(row[1]) in dict_1[int(row[0])]:
                TP += 1
            else:
                FP += 1
        # negative
        else:
            # If document in this negative result is indeed in negative results
            if int(row[1]) in dict_0[int(row[0])]:
                TN += 1
            else:
                FN += 1
    # precision , recall
    # https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
    print("TP: " + str(TP) + ", TN: " + str(TN) + ", FP: " + str(FP) + ", FN: " + str(FN) )
    return [( (TP)/(TP+FP) ),( (TP)/(TP+FN) )]


#print(expected_scores)
#print(scores_sorted)
#print(rbo(expected_scores, scores_sorted))
print("precision, recall: " + str(precision_recall(dev_data_0,dev_data_1,output_binary_label_unsorted)))

GPU 0: Tesla K80 (UUID: GPU-4635fc42-34a9-433b-f323-bc9349cdf07d)
TP: 1472, TN: 1803, FP: 764, FN: 960
precision, recall: [0.6583184257602862, 0.6052631578947368]


In [ ]:
# Trains on full dataset training_data.csv
#cross-encoder/qnli-distilroberta-base, epoch = 1, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/training_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
                #TODO: verwijderen als we volledige dataset inlezen
              dev_data_dict.append(row)


csv.field_size_limit(991072)
to_test = []
temp = 1
with open('./drive/MyDrive/data/test_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
              to_test.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[1]),str(row[2])],label=int(row[5]) ) )
    pairs.append([str(row[1]),str(row[2])])
    labels.append(int(row[5]))

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('cross-encoder/qnli-distilroberta-base', max_length=256)


# Does this need more training?
model.fit(train_dataloader=train_dataloader, epochs=1, warmup_steps=1000,save_best_model=True,output_path="./drive/MyDrive/modelOutput/Full",evaluator=cross_encoder.evaluation.CEBinaryClassificationEvaluator(pairs,labels) )



In [ ]:
#Runs model generated by the cell above this one on test_data.csv
#cross-encoder/qnli-distilroberta-base, epoch = 1, max_length=256, eval = CEBinaryClassificationEvaluator
import sentence_transformers.cross_encoder.evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses, CrossEncoder, cross_encoder, evaluation
from torch.utils.data import DataLoader
import csv
import math

!nvidia-smi -L

# TODO: verwijderen als we volledige dataset inlezen
temp = 1

#Read data, list contains more lists: [query,document_text,relevant]
dev_data_dict = []

dev_data_0 = {}
dev_data_1 = {}

# https://docs.python.org/3/library/csv.html
csv.field_size_limit(991072)
with open('./drive/MyDrive/data/training_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
                #TODO: verwijderen als we volledige dataset inlezen
              dev_data_dict.append(row)


csv.field_size_limit(991072)
to_test = []
temp = 1
with open('./drive/MyDrive/data/test_data.csv', newline='', encoding="utf8") as csvfile:
    dev_data_csv = csv.reader(csvfile, delimiter=',')
    for row in dev_data_csv:
        if row[0] != "Query_number":
              to_test.append(row)

#Define your train examples. (Trains on first temp rows of dataset)
train_examples = []
pairs = []
labels = []
for row in dev_data_dict:
    #TODO: in the data where results are rated relevant or not, their order is also from most to least relevant.
    # We should take this into account by giving a small penalty to label based on position
    train_examples.append( InputExample( texts=[str(row[1]),str(row[2])],label=int(row[5]) ) )
    pairs.append([str(row[1]),str(row[2])])
    labels.append(int(row[5]))

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# Crossencoders are used to predict how well a query matches the result, which is what we need
model = CrossEncoder('./drive/MyDrive/modelOutput/Full', max_length=256)


# Does this need more training?

# TODO: Read data to make predictions about (list of lists each containing [query,document_text]

# TODO: actually use the correct data here
# Make predictions (scores is a list of integers
scores = model.predict([[row[2], row[3]] for row in to_test])

output_label_unsorted = [[to_test[i][0], to_test[i][2],to_test[i][1],to_test[i][3], scores[i]] for i in range(len(to_test))]

with open('./drive/MyDrive/dataOutput/Full.csv', 'w+', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    header = ["Query_number","doc_number","Query","doc_text","label"]
    writer.writerow(header)
    for row in output_label_unsorted:
      # write the data
      writer.writerow(row)


GPU 0: Tesla K80 (UUID: GPU-4a7e8468-3f83-f029-51c8-9c11b50ab20e)
